# Silicon Valley Rent Segmentation

## Introduction

The Silicon Valley is one of the world's hub for technology and innovation. From long time companies like Intel and Hewlett-Packard, social media giants like Google, Facebook and Twitter to a plethora of startups - the Silicon Valley attracts high-skilled workers from all over the world.

When there is an influx of workforce into a region, the cost of living begins to increase. The Silicon Valley is currently going through a 'housing crisis' - with housing prices skyrocketing, affordability is a major issue. For someone moving to the Silicon Valley for a new job, buying a home might far from reach. The other option is finding a place to rent.

## Business Problem

In this study, we will be looking at how median rent for a two-bedroom apartment varies within each zip code of the Santa Clara County. For those who are new and moving to the area, it is best to know where the house rents are relatively low but also closer to social venues like restaurants, theatres, malls, parks, etc. This analysis plans to address this issue.

## Data Requirement and Collection

#### 1. A list of zip codes within the Santa Clara County
This data was scrapped from Santa Clara County Public Health website: https://www.sccgov.org/sites/phd/hi/hd/Pages/city-profiles.aspx
#### 2. Median rent for Santa Clara County by Zip code
This data was scrapped from a real estate search website: https://www.zillow.com/santa-clara-county-ca/home-values/
#### 3. Geo Locator for geographical coordinates of the county
Importing Nominatim to get the coordinates for Santa Clara County 
#### 4. List of venues in each neighborhood
A list of venues in each neighborhood will be pulled from Foursquare API
#### 5. Map generation
Using Folium to generate maps in Python